# 웹 크롤링/스크래핑

## 웹 페이지 추출

### urllib을 이용한 웹페이지 추출

In [30]:
from urllib.request import urlopen

In [31]:
f = urlopen('http://hanbit.or.kr')

In [32]:
# urlopen() 함수는 HTTPResponse 자료형의 객체를 반환
# 파일 객체이므로 open() 함수로 반환되는 파일 객체처럼 핸들링
type(f)

http.client.HTTPResponse

In [33]:
# read() 메서드로 HTTP 응답 본문(bytes 자료형)을 추출
# HTTP 연결은 자동으로 close 되므로 별도의 close() 함수를 호출
f.read()

b"<meta http-equiv='Refresh' content='0; URL=http://www.hanbit.or.kr'>"

In [34]:
f.status

200

In [35]:
f.getheader('Content-Type')

'text/html; charset=UTF-8'

## HTTP 헤더에서 인코딩 방식 추출

In [40]:
import sys
from urllib.request import urlopen

In [41]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [42]:
# HTTP 헤더를 기반으로 인코딩 방식을 추출(명시돼 있지 않을 경우 utf-8을 사용)
encoding = f.info().get_content_charset(failobj='utf-8')

In [43]:
# 인코딩 방식을 표준 오류에 출력
print('encoding: ', encoding)

encoding:  utf-8


In [44]:
# 추출한 인코딩 방식으로 디코딩
text = f.read().decode(encoding)

In [45]:
# 웹 페이지의 내용을 표준 출력에 출력
print(text)

<!DOCTYPE html>
<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>한빛출판네트워크</title>
<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> 
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta property="og:type" content="website"/>
<meta property="og:title" content="한빛출판네트워크"/>
<meta property="og:description" content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀"/>
<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />
<meta property="og:url" content="https://www.hanbit.co.kr/store/books/full_book_list.html"/>
<link rel="canonical" href="https://www.hanbit.co.kr/store/books/full_book_list.html" />
<meta name="keywords" content="한빛미디어,한빛아카데미,한빛비즈,한빛라이프,한빛에듀,리얼타임,대관서비스,책,출판사,IT전문서,IT활용서,대학교재,경제경영,어린이/유아,실용/여행,전자책,자격증,교육,세미나,강의,ebook,정보교과서" />
<meta name="description" content="더 넓은 세

## meta 태그에서 인코딩 방식 추출

In [46]:
import re
import sys
from urllib.request import urlopen

In [47]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')
# bytes 자료형의 응답 본문을 일단 변수에 저장
bytes_content = f.read()

In [48]:
# charset은 HTML의 앞부분에 적혀 있는 경우가 많으므로
# 응답 본문의 앞부분 1024바이트를 ASCII 문자로 디코딩
# ASCII 범위 이외의 문자는 U+FFFD(REPLACEMENT CHARACTER)로 변환되어 예외가 발생하지 않음
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

In [52]:
scanned_text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>������������������������</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="������������������������"/>\r\n<meta property="og:description" content="��� ������ ������, ��� ������ ��������� ������ ��������� ������ ������������ :: ���������������, ������������������, ������������, ���������������, ������������"/>\r\n<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />\r\n<meta property="og:url" content="https://www.hanbit.co.kr/store/books/full_book_list.html"/>\r\n<link rel="canonical" href="https://www.hanbit.co.kr/store/books/full_book_list.html" />\r\n<meta

In [49]:
# 디코딩한 문자열에서 정규 표현식으로 charset 값을 추출
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)

In [50]:
match.group(1)

'utf-8'

In [51]:
if match:
    encoding = match.group(1)
else:
    # charset이 명시돼 있지 않으면 UTF-8을 사용
    encoding = 'utf-8'

## SQLite3 DBMS로 저장

In [53]:
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [54]:
DB_NAME = 'top_cities.db'
TABLE_NAME = 'TOP_CITIES'

In [55]:
def db_save(df, db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            df.to_sql(name = table_name, con=con, index=False, if_exists='append')
            # if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(df), '건 저장완료')

In [56]:
def db_select(db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con=con)
        except Exception as e:
            print(str(e))
        return df

In [57]:
def db_delete(db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM {}'.format(table_name)
            cur.execute(sql)
        except Exception as e:
            print(str(e))

# 웹 크롤러

## 목록 페이지에서 퍼머 링크 목록 추출

In [3]:
import requests
import lxml.html

In [4]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('.view_box a'):
    url = a.get('href')
    print(url)

/store/books/look.php?p_code=B9143267027
javascript:;
/store/books/look.php?p_code=B9143267027
/store/books/look.php?p_code=B3517216657
javascript:;
/store/books/look.php?p_code=B3517216657
/store/books/look.php?p_code=B4872223435
javascript:;
/store/books/look.php?p_code=B4872223435
/store/books/look.php?p_code=B9279330130
javascript:;
/store/books/look.php?p_code=B9279330130
/store/books/look.php?p_code=B6787201225
javascript:;
/store/books/look.php?p_code=B6787201225
/store/books/look.php?p_code=B3963656224
javascript:;
/store/books/look.php?p_code=B3963656224
/store/books/look.php?p_code=B1740090592
javascript:;
/store/books/look.php?p_code=B1740090592
/store/books/look.php?p_code=B2672932319
javascript:;
/store/books/look.php?p_code=B2672932319
/store/books/look.php?p_code=B3872657116
javascript:;
/store/books/look.php?p_code=B3872657116
/store/books/look.php?p_code=B8226621719
javascript:;
/store/books/look.php?p_code=B8226621719
/store/books/look.php?p_code=B9479195027
javascrip

## 상대 URL을 절대 URL로 변환

In [6]:
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

https://www.hanbit.co.kr/store/books/look.php?p_code=B9143267027
https://www.hanbit.co.kr/store/books/look.php?p_code=B3517216657
https://www.hanbit.co.kr/store/books/look.php?p_code=B4872223435
https://www.hanbit.co.kr/store/books/look.php?p_code=B9279330130
https://www.hanbit.co.kr/store/books/look.php?p_code=B6787201225
https://www.hanbit.co.kr/store/books/look.php?p_code=B3963656224
https://www.hanbit.co.kr/store/books/look.php?p_code=B1740090592
https://www.hanbit.co.kr/store/books/look.php?p_code=B2672932319
https://www.hanbit.co.kr/store/books/look.php?p_code=B3872657116
https://www.hanbit.co.kr/store/books/look.php?p_code=B8226621719
https://www.hanbit.co.kr/store/books/look.php?p_code=B9479195027
https://www.hanbit.co.kr/store/books/look.php?p_code=B8945183661
https://www.hanbit.co.kr/store/books/look.php?p_code=B8460934222
https://www.hanbit.co.kr/store/books/look.php?p_code=B1191734987
https://www.hanbit.co.kr/store/books/look.php?p_code=B2082711299
https://www.hanbit.co.kr/

## URL 목록 추출을 위한 scrape_list_page() 함수 정의

In [17]:
import time

def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url) # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response) # 상세 페이지에서 상세 정보를 추출
        print(ebook) # 상세 정보 출력
        break

In [11]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

## 상세 페이지 스크래핑

In [21]:
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3'),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
                   for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                   if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [24]:
import re

def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [25]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B9143267027', 'title': [<Element h3 at 0x2203b5a7bd0>], 'price': '16,650', 'content': ['프롤로그│학문과 삶의 접점을 이야기하는 인문학', '1강 르네상스 미술의 한 장면│이화진', '왜 알아야 할까', '1교시 ｜ 피렌체의 상인들', '2교시 ｜ 하늘을 향한 둥근 지붕', '3교시 ｜ 다윗은 어떻게 조각되었나', '4교시 ｜ 열린 창으로 바라본 세계', '5교시 ｜ 바티칸의 영광, 교황들의 찬가', '2강 천문이 곧 인문이다│안나미', '왜 알아야 할까', '1교시 ｜ 별이 알려주는 내 운명, 점성술', '2교시 ｜ 동양의 하늘 vs. 서양의 하늘', '3교시 ｜ 불길한 별의 꼬리, 혜성', '4교시 ｜ 태양 기록의 비과학과 과학', '5교시 ｜ 죽어야 다시 태어나는 별, 초신성', '3강 지도를 가진 자, 세계를 제패하다│이정선', '왜 알아야 할까', '1교시 ｜ 고지도의 매력과 유혹', '2교시 ｜ 한눈에 보는 세계지도의 역사', '3교시 ｜ 탐험의 시작, 미지의 세계를 향하다', '4교시 ｜ 지도상 바다 명칭의 유래와 우리 바다 ‘동해’', '5교시 ｜ <대동여지도>, 조선의 네트워크를 구축하다', '4강 나를 찾아가는 글쓰기│최옥정', '왜 알아야 할까', '1교시 ｜ 말과 글이 삶을 바꾼다', '2교시 ｜ 독서, 글쓰기에 연료를 공급하는 일', '3교시 ｜ 소설가의 독서법', '4교시 ｜ 어쨌든 문장이다', '5교시 ｜ 마음을 다잡는 글쓰기의 기술', '5강 클래식, 문학을 만나다│나성인', '왜 알아야 할까', '1교시 ｜ 작곡가의 상상 속에 녹아든 괴테의 문학 : <파우스트>', '2교시 ｜ 셰익스피어의 언어, 음악이 되다 : <한여름 밤의 꿈>', '3교시 ｜ 자유를 갈망하는 시대정신의 증언자, 빅토르 위고 : <리골레토>', '4교시 ｜ 신화의 해석, 혁명의 서막 

## 고급 웹 크롤러
### daum 뉴스

In [26]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [27]:
REG_DATE = '20200819'

In [30]:
response = requests.get('https://news.daum.net/breakingnews/digital?regDate={}'.format(REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200819230943259 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230922257 트럼프 "오라클은 대단한 회사"..틱톡 인수 지지
https://v.daum.net/v/20200819230858252 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230757249 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230752248 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230732247 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230639242 한국심리학회 연차학술대회 20~22일 열려
https://v.daum.net/v/20200819230626241 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230623240 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230557231 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230516226 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230512224 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230448221 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819225027087 글로벌창업사

In [31]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [32]:
REG_DATE='20200901'

In [40]:
response = requests.get('https://news.daum.net/breakingnews/digital?regDate={}'.format(REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200901235714436 갤럭시Z폴드2 톰브라운 에디션
https://v.daum.net/v/20200901235712435 베일 벗은 갤럭시Z폴드2
https://v.daum.net/v/20200901235711434 삼성전자 갤럭시Z폴드2 공개
https://v.daum.net/v/20200901235709433 삼성전자 차세대 주력 스마트폰 갤럭시Z폴드2 공개
https://v.daum.net/v/20200901235707432 삼성 차세대 주력 스마트폰 갤럭시Z폴드2 공개
https://v.daum.net/v/20200901235706431 갤럭시Z폴드2 공개한 삼성
https://v.daum.net/v/20200901235704430 갤럭시Z폴드2 공개 '스마트폰 시장 변화는?'
https://v.daum.net/v/20200901235450416 갤럭시Z폴드2, 화면 키우고 멀티태스킹 강화..가격은 그대로(종합)
https://v.daum.net/v/20200901235418413 삼성 갤럭시Z 폴드2, 베일 벗었다.. 239만8000원
https://v.daum.net/v/20200901234659387 "폴더블 새 역사 쓴다"..베일 벗은 '갤럭시Z폴드2'
https://v.daum.net/v/20200901234444371 삼성전자 '갤럭시Z폴드2' 온라인으로 공개
https://v.daum.net/v/20200901234438369 삼성전자 '갤럭시Z폴드2' 온라인으로 공개
https://v.daum.net/v/20200901234434368 삼성전자 '갤럭시Z폴드2' 온라인으로 공개
https://v.daum.net/v/20200901234428367 '삼성 갤럭시Z폴드2' 온라인 공개
https://v.daum.net/v/20200901234418364 '삼성 갤럭시Z폴드2' 온라인 공개


In [41]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text:
            body.append(re.sub(punc, '', p.text))
    full_body = ' '.join(body)
        
    return full_body

get_detail('https://news.v.daum.net/v/20200819230922257')

'지디넷코리아김익현 미디어연구소장 도널드 트럼프 미국 대통령이 오라클의 틱톡 인수에 강한 지지 의사를 드러냈다 CNBC를 비롯한 주요 외신들에 따르면 트럼프는 19일현지시간 미국 애리조나주 유마 유세 현장에서 오라클이 틱톡을 인수하기에 좋은 기업이냐는 질문을 받고 오라클은 대단한 회사라고 답변했다 트럼프는 오라클이 틱톡 인수에 관심을 갖고 있다는 보도가 나온 직후 \xa0이 같은 발언을 했다 트럼프의 예사롭지 않은 발언에 관심이 쏠리는 건 엘리슨의 이런 정치적 성향 때문이다 보도에 따르면 트럼프는 오라클은 대단한 회사이며 오라클 소유주는 엄청난tremendous 사람이라고 생각한다면서 오라클은 틱톡을 잘 다룰 수 있을 것이다고 말했다 트럼프 대통령은 바이트댄스 측에 90일 이내에 틱톡 미국 사업을 떼어내라고 통보했다 이 통보에 따라 중국 기업 바이트댄스는 틱톡 미국 사업부문 등에 대한 매각 작업에 공식 착수했다 그 동안 틱톡 인수 경쟁에선 마이크로소프트MS가 한 발 앞서 있다는 전망이 지배적이었다 하지만 오라클이 틱톡 지분을 갖고 있는 미국 벤처캐피털업체들과 공동으로 인수 경쟁에 뛰어들었다는 보도가 나오면서 새로운 국면에 접어들었다 CNBC는 오라클이 틱톡 인수를 위해 얼마나 준비하고 있는지는 알려지지 않았다고 전했다 바이트댄스 투자자들은 틱톡의 전체 가치를 500억 달러 수준으로 평가하고 있다 김익현 미디어연구소장sini@zdnetcokr'

### daum 뉴스 첫 페이지 목록과 상세 페이지

In [42]:
page=1
max_page=0
REG_DATE='20200819'

In [43]:
response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'.format(page, REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    article = get_detail(url)
    print(f'URL : {url}')
    print(f'TITLE: {a.text}')
    print(f'ARTICLE: {article}')
    print('-'*100)

URL : https://v.daum.net/v/20200819230943259
TITLE: RUSSIA SPACE DOGS BELKA AND STRELKA
ARTICLE: epa08612780 Visitors look at stuffed dogs Belka L and Strelka R at the Museum of Cosmonautics in Moscow Russia 19 August 2020 Russia celebrates the 60th anniversary of KorablSputnik 2 on 19 August The two female dogs Strelka and Belka among other animals the first creatures to return alive from space EPAYURI KOCHETKOV
----------------------------------------------------------------------------------------------------
URL : https://v.daum.net/v/20200819230922257
TITLE: 트럼프 "오라클은 대단한 회사"..틱톡 인수 지지
ARTICLE: 지디넷코리아김익현 미디어연구소장 도널드 트럼프 미국 대통령이 오라클의 틱톡 인수에 강한 지지 의사를 드러냈다 CNBC를 비롯한 주요 외신들에 따르면 트럼프는 19일현지시간 미국 애리조나주 유마 유세 현장에서 오라클이 틱톡을 인수하기에 좋은 기업이냐는 질문을 받고 오라클은 대단한 회사라고 답변했다 트럼프는 오라클이 틱톡 인수에 관심을 갖고 있다는 보도가 나온 직후  이 같은 발언을 했다 트럼프의 예사롭지 않은 발언에 관심이 쏠리는 건 엘리슨의 이런 정치적 성향 때문이다 보도에 따르면 트럼프는 오라클은 대단한 회사이며 오라클 소유주는 엄청난tremendous 사람이라고 생각한다면서 오라클은 틱톡을 잘 다룰 수 있을 것이다고 말했다 트럼프 대통령은 바이트댄스 측에 90일 이내에 틱톡 미국 사업을 떼어

### daum 뉴스 다음페이지 이동과 마지막 페이지

In [52]:
# 페이지 번호 중에서 max 페이지 가져오기
for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
    try:
        num = int(a.text)
        print(num)
        if max_page < num:
            max_page = num
    except:
        pass

# 마지막 페이지 여부 확인
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1

2
3
4
5
6
7
8
9
10


### 크롤링 정보 저장 및 조회

In [53]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name='NEWS_LIST', con=con, index=False, if_exists='append')
            # if_exists : {'fail', 'replace', 'append'} default: fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [55]:
def db_delete():
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [56]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con=con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST

## 셀레늄을 이용한 크롤러

In [57]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


### Webdriver 로드 및 브라우저 가동

In [58]:
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd

In [59]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");

browser = webdriver.Chrome('chromedriver', options=options)

In [61]:
browser.get('https://www.data.go.kr/')
browser.implicitly_wait(5)

* 가동된 브라우저를 통한 URL 접속

In [63]:
browser.find_element_by_xpath('//*[@id="header"]/div/div/div/div[2]/div/a[1]').click()
browser.implicitly_wait(5)

* ID/Password 입력 및 로그인

In [65]:
browser.find_element_by_xpath('//*[@id="mberId"]').send_keys('아이디')
browser.find_element_by_xpath('//*[@id="pswrd"]').send_keys('비밀번호')
browser.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
browser.implicitly_wait(5)

* 정보공유 링크 클릭

In [70]:
browser.find_element_by_xpath('//*[@id="M000400_pc"]/a').click()

* 자료실 링크 클릭

In [71]:
browser.find_element_by_xpath('//*[@id="M000402_pc"]/a').click()

* 자료실 데이터 추출 및 저장

In [73]:
trs = browser.find_elements_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr')
df_list = []
for tr in trs:
    df = pd.DataFrame({
        'NO': [tr.find_element_by_xpath('td[1]').text],
        'TITLE': [tr.find_element_by_xpath('td[2]').text],
        'IQRY': [tr.find_element_by_xpath('td[3]').text],
        'REGDT': [tr.find_element_by_xpath('td[4]').text],
        'CHGDT': [tr.find_element_by_xpath('td[5]').text],
    })
    df_list.append(df)
    
ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)

table NEWS_LIST has no column named NO
10 건 저장완료..


* 자료실 글목록 상세보기 클릭

In [74]:
browser.find_element_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr[1]/td[2]/a').click()
browser.implicitly_wait(3)

* 상세보기 첨부파일 다운로드 및 브라우져 종료

In [75]:
browser.find_element_by_xpath('//*[@id="recsroomDetail"]/div[2]/div[4]/div[1]/a').click()
time.sleep(10)

In [76]:
browser.quit()

* 브라우저를 기동하지 않고 background 작업 수행 - headless 기능 이용

In [77]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1280x1024')

broswer = webdriver.Chrome('chromedriver', options=options)